In [21]:
import numpy as np
import pandas as pd
import yfinance as yf

In [22]:
# tickers = pd.read_csv("TickerMeta.csv")
# tickers["working"] = 0
# tickers["close"] = 0
# tickers["checked"] = 0

tickers = pd.read_parquet("tickers.parquet")

In [23]:
# tickers.to_parquet("tickers.parquet")
# tickers = pd.read_parquet("tickers.parquet")

In [24]:
leftoff = tickers[tickers.checked == 1].index.max()
print(leftoff)

106327


In [25]:
import time

In [26]:
'''
COUNT1 = 0
COUNT2 = 0

for i in tickers.loc[leftoff-1:,:].itertuples():
    COUNT1 += 1
    COUNT2 += 1
    hist = yf.Ticker(i[1]).history(period="1d")
    if hist.empty==False:
        tickers.loc[i[0], "working"] = 1
        tickers.loc[i[0], "close"] = hist["Close"][0]
    tickers.loc[i[0], "checked"] = 1
    if COUNT1%100==0:
        tickers.to_parquet("tickers.parquet")
        print("Saved")
    if COUNT2%800==0:
        time.sleep(30)
        print("Slept")
'''

'\nCOUNT1 = 0\nCOUNT2 = 0\n\nfor i in tickers.loc[leftoff-1:,:].itertuples():\n    COUNT1 += 1\n    COUNT2 += 1\n    hist = yf.Ticker(i[1]).history(period="1d")\n    if hist.empty==False:\n        tickers.loc[i[0], "working"] = 1\n        tickers.loc[i[0], "close"] = hist["Close"][0]\n    tickers.loc[i[0], "checked"] = 1\n    if COUNT1%100==0:\n        tickers.to_parquet("tickers.parquet")\n        print("Saved")\n    if COUNT2%800==0:\n        time.sleep(30)\n        print("Slept")\n'

## Shave

In [27]:
tickers.loc[tickers["Ticker"]=="FB"]

,Ticker,Name,Exchange,Category Name,Country,working,close,checked
8343,FB,"Facebook, Inc.",NMS,Internet Information Providers,USA,0,0.0,1


In [28]:
tickers.loc[tickers["Ticker"]=="FB", "Ticker"] = "META"
tickers.loc[tickers["Ticker"]=="META", "Name"] = "Meta Platforms, Inc"
tickers.loc[tickers["Ticker"]=="META", "working"] = 1

In [30]:
working_tic = tickers[(tickers["working"]==1) &
                    #   (tickers["close"]>0) & 
                      ~(tickers["Exchange"].isna())
                      ].copy()

In [32]:
exch_order = ["NMS", "NASDAQ", "NYSE", "TSX", "PNK"]
tick_order = ["AMZN","AAPL","MSFT","GOOG","META","NVDA","PG","NFLX","V"]  

In [33]:
exch_order = exch_order + list(set(working_tic.Exchange.unique().tolist()) - set(exch_order))
tick_order = tick_order + list(set(working_tic.Ticker.unique().tolist()) - set(tick_order))

In [34]:
pd.Series(tick_order).value_counts()

AMZN            1
077970.KS       1
EAT             1
VYCO            1
CCYY            1
               ..
2385.TW         1
TELECANOR.BO    1
FAR.TO          1
UNCFF           1
5704.TWO        1
Length: 42099, dtype: int64

In [35]:
working_tic['Exchange'] = pd.Categorical(working_tic['Exchange'], categories=exch_order, ordered=True)
working_tic['Ticker'] = pd.Categorical(working_tic['Ticker'], categories=tick_order, ordered=True)

# Sort by "Ticker", "Exchange", and then by "Close"
df_sorted = working_tic.sort_values(by=["Ticker", "Exchange", "close"], ascending=[True, True, False])

In [36]:
df_sorted[df_sorted["Ticker"]=="META"]

,Ticker,Name,Exchange,Category Name,Country,working,close,checked
8343,META,"Meta Platforms, Inc",NMS,Internet Information Providers,USA,1,0.0,1


In [37]:
# df_sorted.to_parquet("tickers_s.parquet")